## Linear Regression with PyTorch

#### Part 2 of "PyTorch: Zero to GANs"

*This post is the second in a series of tutorials on building deep learning models with PyTorch, an open source neural networks library developed and maintained by Facebook. Check out the full series:*

1. [PyTorch Basics: Tensors Operations](https://www.kaggle.com/erdemuysal/pytorch-tensor-operations-for-beginners)
2. [Linear Regression from Scatch and with PyTorch using Gradient Descent](https://www.kaggle.com/erdemuysal/linear-regression-from-scratch-and-with-pytorch/notebook)

Continuing where the [previous tutorial](https://www.kaggle.com/erdemuysal/pytorch-tensor-operations-for-beginners) left off, we'll discuss one of the foundational  algorithms of machine learning in this post: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Our data is called California Housing Prices.

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :
```
house_price  = w11 * location + w12 * number_of_rooms + w13 * square_feet + b1
```
Visually, it means that the house_price is a linear or planar function of location, number_of_rooms and square_feet:

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.

## Imports and Definitions

We begin by importing necessery libraries and frameworks:

In [1]:
# Uncomment the command below if Numpy or PyTorch is not installed
# !conda install numpy pytorch cpuonly -c pytorch -y

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

In [3]:
# Hyperparameters
split_rate = 0.2
batch_size = 64
learning_rate = 1e-4

# Other constants
DATA_FILENAME = 'housing.csv'
TARGET_COLUMN = 'median_house_value'  # Target to predict

## Data Preparation

Let us first load the dataset as Pandas data frame.

In [4]:
df = pd.read_csv('../input/california-housing-prices/housing.csv')
df.head()  # Print first 5 row of the data frame, 5 is default value but desired value can be passed as an argument

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
df.columns, len(df.columns)

(Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
        'total_bedrooms', 'population', 'households', 'median_income',
        'median_house_value', 'ocean_proximity'],
       dtype='object'),
 10)

For appropriate predictions, we should remove some of the entries like infinite, nan and non-numerical values.

In [6]:
df.replace([np.inf, -np.inf], np.nan)  # Replace inf values with NaNs
df.dropna(inplace=True)  # Drop NaN values from the dataframe

In [7]:
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


The training data can be represented using two matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [9]:
# Inputs: (longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income)
# Convert from Pandas dataframe to numpy arrays
inputs = df.drop([TARGET_COLUMN, 'ocean_proximity'], axis=1).to_numpy()  # Drop TARGET_COLUMN since it is target, can not be used as input
targets = df[[TARGET_COLUMN]].to_numpy()                                 # Drop 'ocean_proximity' since it is not numerical value
inputs.shape, targets.shape

((20433, 8), (20433, 1))

In [10]:
input_size = inputs.shape[1]  # 8
output_size = targets.shape[1]  # 1
input_size, output_size

(8, 1)

We have separated the input and target variables, because we are going to operate on them separately. Also, we have created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors. Since PyTorch only be able to perform operations using tensors, we should convert all input and target arrays to tensors and create a tensor dataset. 

In [11]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

We are normalizing the data for better results.

In [12]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  # Noramalize
#targets = targets / targets_mean  # Normalize

## Linear Regression Model from Scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the target variable median house value.

In [13]:
w = torch.randn(output_size, input_size, requires_grad=True)  # Weights
b = torch.randn(output_size, requires_grad=True)  # Biases
w, w.shape, b, b.shape

(tensor([[ 1.2403, -0.7097,  1.0364,  0.6800,  0.0442,  1.6242, -0.5536,  1.0774]],
        requires_grad=True),
 torch.Size([1, 8]),
 tensor([1.7892], requires_grad=True),
 torch.Size([1]))

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean of 0 and standard deviation of 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [14]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `t()` method returns the transpose of a tensor. The matrix obtained by passing the input data into the model is a set of predictions for the target variables. 

Let's compare the predictions of our model with the actual targets.

In [15]:
# Generate predictions
predictions = model(inputs)
predictions, targets # Compare with targets

(tensor([[6.5685],
         [8.7706],
         [6.9684],
         ...,
         [4.6388],
         [4.4046],
         [5.2690]], grad_fn=<AddBackward0>),
 tensor([[452600.],
         [358500.],
         [352100.],
         ...,
         [ 92300.],
         [ 84700.],
         [ 89400.]]))

You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to *just work*.

## Loss Function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

* Calculate the difference between the two matrices (`predictions` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [16]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff ** 2) / diff.numel()

`torch.sum` returns the sum of all the elements in a tensor, and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

In [17]:
# Compute loss
loss = mse(predictions, targets)
loss

tensor(5.6115e+10, grad_fn=<DivBackward0>)

Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by about 145 (square root of the loss 20834)*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the *loss*, because it indicates how bad the model is at predicting the target variables. Lower the loss, better the model.

## Compute Gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases, because they have `requires_grad` set to `True`.

In [18]:
# Compute gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [19]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 1.2403, -0.7097,  1.0364,  0.6800,  0.0442,  1.6242, -0.5536,  1.0774]],
       requires_grad=True)
tensor([[-413892.0312, -411714.4375, -424521.8750, -439220.0000, -422701.0000,
         -409069.4375, -425182.3125, -491683.0938]])


The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the [slope](https://en.wikipedia.org/wiki/Slope) of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:
* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:
* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling `.zero_()` method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results.

In [20]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([0.])


## Adjust Weights and Biases Using Gradient Descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [21]:
# Generate predictions
predictions = model(inputs)
predictions

tensor([[6.5685],
        [8.7706],
        [6.9684],
        ...,
        [4.6388],
        [4.4046],
        [5.2690]], grad_fn=<AddBackward0>)

Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.

In [22]:
# Calculate the loss
loss = mse(predictions, targets)
loss

tensor(5.6115e+10, grad_fn=<DivBackward0>)

In [23]:
# Compute gradients
loss.backward()
w.grad, b.grad

(tensor([[-413892.0312, -411714.4375, -424521.8750, -439220.0000, -422701.0000,
          -409069.4375, -425182.3125, -491683.0938]]),
 tensor([-413716.2812]))

Finally, we update the weights and biases using the gradients computed above.

In [24]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * learning_rate
    b -= b.grad * learning_rate
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

* We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases. 

* We multiply the gradients with a really small number (`10^-5` in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the *learning rate* of the algorithm. 

* After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.

In [25]:
w, b

(tensor([[42.6295, 40.4617, 43.4885, 44.6020, 42.3143, 42.5311, 41.9646, 50.2457]],
        requires_grad=True),
 tensor([43.1608], requires_grad=True))

With the new weights and biases, the model should have lower loss.

In [26]:
# Calculate loss
predictions = model(inputs)
loss = mse(predictions, targets)
loss

tensor(5.5950e+10, grad_fn=<DivBackward0>)

We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent.

## Train for Multiple Epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [27]:
# Train for 1000 epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * learning_rate
        b -= b.grad * learning_rate
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [28]:
# Calculate loss
predictions = model(inputs)
loss = mse(predictions, targets)
loss

tensor(1.4193e+10, grad_fn=<DivBackward0>)

As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets.

In [29]:
# Predictions
predictions

tensor([[163203.4219],
        [268699.8125],
        [172851.0312],
        ...,
        [136624.5000],
        [128510.8281],
        [154556.9688]], grad_fn=<AddBackward0>)

In [30]:
# Targets
targets

tensor([[452600.],
        [358500.],
        [352100.],
        ...,
        [ 92300.],
        [ 84700.],
        [ 89400.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs. 

## Linear Regression Using PyTorch Built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

## Splitting Data into Batches of Data

We already created a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

In [31]:
# Define dataset
dataset = TensorDataset(inputs, targets)
dataset[0:2]

(tensor([[1.0222, 1.0631, 1.4319, 0.3338, 0.2398, 0.2260, 0.2523, 2.1506],
         [1.0222, 1.0625, 0.7334, 2.6926, 2.0563, 1.6850, 2.2786, 2.1444]]),
 tensor([[452600.],
         [358500.]]))

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [32]:
# Define data loader
train_loader = DataLoader(dataset, batch_size, shuffle=True)

The data loader is typically used in a `for-in` loop. Let's look at an example.

In [33]:
for xb, yb in train_loader:
    print(xb)
    print(yb)
    break

tensor([[0.9874, 0.9514, 1.3621, 0.9490, 1.0151, 1.2976, 1.0372, 1.0035],
        [1.0252, 1.0706, 0.3143, 0.4938, 0.3291, 0.4786, 0.3804, 1.9374],
        [1.0159, 1.0546, 0.4889, 0.6046, 0.5429, 0.6225, 0.5747, 1.2044],
        [1.0174, 1.0291, 0.9779, 0.7305, 1.0337, 1.2050, 1.0712, 0.5008],
        [0.9898, 0.9556, 1.8161, 0.7476, 0.7697, 0.7474, 0.8189, 0.9412],
        [0.9796, 0.9303, 0.5588, 1.4819, 1.2475, 1.2344, 1.3275, 0.8208],
        [0.9891, 0.9505, 0.7334, 2.3167, 2.1009, 2.2766, 2.2285, 1.1048],
        [0.9839, 0.9438, 0.3492, 1.8259, 1.1955, 1.2688, 1.1773, 2.2727],
        [1.0213, 1.0580, 1.4319, 1.3256, 1.4632, 1.2976, 1.5017, 0.6920],
        [0.9861, 0.9477, 0.9080, 2.1073, 2.5991, 3.1896, 2.6690, 0.7983],
        [1.0202, 1.0535, 0.6286, 1.3594, 1.3293, 1.1741, 1.3235, 1.0189],
        [1.0238, 1.0591, 1.8161, 0.5841, 0.6879, 0.7972, 0.7268, 1.1140],
        [0.9853, 0.9544, 0.4191, 3.4883, 3.0528, 3.0401, 3.1115, 1.2810],
        [1.0225, 1.0743, 0.8382, 1.073

In each iteration, the data loader returns one batch of data, with the given batch size. If `shuffle` is set to `True`, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [34]:
# Define model
model = nn.Linear(input_size, output_size)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.3171,  0.1518,  0.1701, -0.1674,  0.2445, -0.0471,  0.2284, -0.0842]],
       requires_grad=True)
Parameter containing:
tensor([0.1285], requires_grad=True)


PyTorch models also have a helpful `.parameters` method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [35]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3171,  0.1518,  0.1701, -0.1674,  0.2445, -0.0471,  0.2284, -0.0842]],
        requires_grad=True),
 Parameter containing:
 tensor([0.1285], requires_grad=True)]

We can use the model to generate predictions in the exact same way as before:

In [36]:
# Generate predictions
predictions = model(inputs)
predictions

tensor([[0.7263],
        [1.0513],
        [0.8228],
        ...,
        [0.9241],
        [0.8875],
        [0.9606]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss` from `torch.nn.functional`. The `torch.nn.functional` package contains many useful loss functions and several other utilities. 

In [37]:
# Define loss function
criterion = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [38]:
loss = criterion(model(inputs), targets)
loss

tensor(5.6117e+10, grad_fn=<MseLossBackward>)

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [39]:
# Define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Note that `model.parameters()` is passed as an argument to `optim.SGD`, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.

## Train the Model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

In [40]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update, and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [41]:
fit(100, model, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 7307639296.0000
Epoch [20/100], Loss: 4309603840.0000
Epoch [30/100], Loss: 8685913088.0000
Epoch [40/100], Loss: 9940669440.0000
Epoch [50/100], Loss: 7668150272.0000
Epoch [60/100], Loss: 11361121280.0000
Epoch [70/100], Loss: 9915807744.0000
Epoch [80/100], Loss: 4636130816.0000
Epoch [90/100], Loss: 16285785088.0000
Epoch [100/100], Loss: 9558606848.0000


Let's generate predictions using our model and verify that they're close to our targets.

In [42]:
# Generate predictions
predictions = model(inputs)
predictions

tensor([[367141.8438],
        [371599.1562],
        [347588.4688],
        ...,
        [128728.6562],
        [135276.8438],
        [149707.6875]], grad_fn=<AddmmBackward>)

In [43]:
# Compare with targets
targets

tensor([[452600.],
        [358500.],
        [352100.],
        ...,
        [ 92300.],
        [ 84700.],
        [ 89400.]])

Indeed, the predictions are quite close to our targets, and now we have a fairly good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall and humidity in a region.

## Further Reading

We've covered a lot of ground this this tutorial, including *linear regression* and the *gradient descent* optimization algorithm. Here are a few resources if you'd like to dig deeper into these topics:

* For a more detailed explanation of derivates and gradient descent, see [these notes from a Udacity course](https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf). 

* For an animated visualization of how linear regression works, [see this post](https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09).

* For a more mathematical treatment of matrix calculus, linear regression and gradient descent, you should check out [Andrew Ng's excellent course notes](https://github.com/Cleo-Stanford-CS/CS229_Notes/blob/master/lectures/cs229-notes1.pdf) from CS229 at Stanford University.

* To practice and test your skills, you can participate in the [Boston Housing Price Prediction](https://www.kaggle.com/c/boston-housing) competition on Kaggle, a website that hosts data science competitions.

## Reference Links
* Official documentation for torch.Tensor: https://pytorch.org/docs/stable/tensors.html
* PyTorch Zero to GANs Course: http://zerotogans.com/
* https://jovian.ml/aakashns/02-linear-regression
* https://hackernoon.com/visualizing-linear-regression-with-pytorch-9261f49edb09

Special thanks to Jovian.ml team and freeCodeCamp.org!

With this, we complete our discussion of linear regression in PyTorch, and we’re ready to move on to the next topic: *Logistic regression*.